# Analysis of Experimental Data

Most of the analysis is the same for experiments and simulations.
The comparison across conditions is implemented in a stand-alone script `ana/process_conditions.py`, that can be run from a terminal and takes a the following arguments:
* `-i` the base path to the folder where the data is stored.
* `-t` the type of experiment (yields the right subfolders and file names):
    - `exp` for main results, from optogenetic stimulation and different topologies
    - `exp_chemical` for the experiments with KCl
    - `exp_bic` for the experiments with Bicuculline
    - `sim_partial` for simulations where only part of the system was targeted.
* `-o` where to store the output path.

To create the preprocessed data, navigate to the base directory and run:
```bash
python ./ana/process_conditions.py -t exp -i ./dat/experiments/raw/  -o ./dat/experiments/processed/
python ./ana/process_conditions.py -t exp_chemical -i ./dat/experiments/raw/ -o ./dat/experiments/processed/
python ./ana/process_conditions.py -t exp_bic -i ./dat/experiments/raw/ -o ./dat/experiments/processed/
```


This should yield the following files:
```bash
>>> tree -L 2 --dirsfirst ./dat/experiments/processed/
dat/experiments/processed/
├── 1b
│   ├── 210315_A
│   ├── 210315_C
│   ├── 210405_C
│   ├── 210406_B
│   ├── 210406_C
│   ├── 210719_B
│   ├── 210719_C
│   └── 210726_B
├── 3b
│   ├── 210316_A
│   ...
├── Bicuculline_1b
│   ├── 210907_1bB
│   ...
├── KCl_1b
│   ├── 210420_C
│   ...
├── merged
│   ├── 210401_A
│   ...
├── 1b.hdf5
├── 3b.hdf5
├── Bicuculline_1b.hdf5
├── KCl_1b.hdf5
└── merged.hdf5
```

where the `*.hdf5` files contain the preprocessed data and the folders for each experiment have some additional info. See also `save_analysed_h5f` in `ana/process_conditions.py`.

Low-level plotting functions are contained in `ana/plot_helper.py` and
the higher-level wrappers as well as further analysis are in `ana/paper_plots.py`.
In particular, most contend of this notebook can also be found in `paper_plots.py/fig_x()`

Experiments are depicted in Figures 1 and 2, and in the Supplemental Material.
For fine-grained control, we produced every figure panel as a stand-alone and combined them later.

# Plotting

In [ ]:
# The autoreload extension allows you to tweak the code in the imported modules (`pp`)
# and rerun cells to reflect the changes.
%load_ext autoreload
%autoreload 2
%load_ext ipy_dict_hierarchy
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


import sys
sys.path.append("../ana/")

import paper_plots as pp
# reduce the printed output, we have lots of details on the INFO level.
pp.log.setLevel("ERROR")

In [ ]:
print(pp.fig_1.__doc__)
pp.fig_1()

In [ ]:
print(pp.fig_2.__doc__)
pp.fig_2()

# Supplemental Material


```bash
python ./ana/process_conditions.py -t sim_partial -i ./dat/simulations/lif/raw/ -o ./dat/simulations/lif/processed/partial/
```


In [ ]:
print(pp.sm_exp_trialwise_observables.__doc__)
pp.sm_exp_trialwise_observables(prefix=f"{pp.p_fo}/exp_layouts_sticks")

# the test results are logged at INFO level and
# returned as a pandas dataframe.
# pp.log.setLevel("INFO")
print("p-values: two-sided, paired-sample t-test:")
pp.nhst_pairwise_for_trials(
    observables=[
        # "Mean Correlation",
        # "Mean IBI",
        "Mean Fraction", # this is the event size
        "Functional Complexity",
        # "Mean Core delays",
        # "Mean Rate",
        # "Median IBI",
        # "Median Core delays",
    ],
    layouts=["1b", "3b", "merged"],
)

In [ ]:
from bayesian import best, best_paired, probability_of_direction
from paper_plots import p_exp, p_fo, p_sim, _p_base

In [ ]:
chem = pp.load_pd_hdf5(f"{p_exp}/processed/KCl_1b.hdf5")
opto = pp.load_pd_hdf5(f"{p_exp}/processed/1b.hdf5")

dfs = dict()
dfs["exp"] = opto["trials"].query("`Condition` in ['pre', 'stim']")
dfs["exp_chemical"] = chem["trials"]

df = dfs["exp"]
df_on = df.query("`Stimulation` == 'On'")
df_off = df.query("`Stimulation` == 'Off'")
trials = df_on.Trial.unique()
print(f"trials are consistent: {np.all([t in df_off.Trial.unique() for t in trials])}")

obs = "Functional Complexity"

# build the pairs
y1 = []
y2 = []
for t in trials:
    val1 = df_off.query(f"`Trial` == '{t}'")[obs].values
    assert len(val1) == 1
    val2 = df_on.query(f"`Trial` == '{t}'")[obs].values
    assert len(val2) == 1

    y1.append(val1[0])
    y2.append(val2[0])

# trace = best(y1, y2, n1="pre", n2="stim")
diff_trace = best_paired(y1, y2, draws=4000, chains=4)
diff_trace

In [ ]:
# import arviz as az
# import bokeh
# bokeh.io.output_notebook() 

# az.plot_posterior(
#     trace,
#     var_names=["diff_of_means", "diff_of_stds", "effect_size"],
#     # reference 0 for no effect
#     ref_val=0,
#     backend="bokeh",
# );

az.plot_posterior(
    diff_trace,
    var_names=["mean_of_diffs", "std_of_diffs", "effect_size"],
    # reference 0 for no effect
    ref_val=0,
    backend="bokeh",
);
diff_trace